In [3]:
! pip install -q tensorflow numpy pandas scikit-learn mlxtend dataprep transformers
! cp drive/My\ Drive/Colab\ Notebooks/*.csv ./
! cp drive/My\ Drive/Colab\ Notebooks/*.pkl ./

     |████████████████████████████████| 92kB 3.4MB/s 
     |████████████████████████████████| 778kB 17.8MB/s 
     |████████████████████████████████| 512kB 37.5MB/s 
     |████████████████████████████████| 5.5MB 28.3MB/s 
     |████████████████████████████████| 2.2MB 46.9MB/s 
     |████████████████████████████████| 71kB 6.9MB/s 
     |████████████████████████████████| 1.4MB 44.1MB/s 
     |████████████████████████████████| 8.6MB 15.3MB/s 
     |████████████████████████████████| 368kB 44.7MB/s 
     |████████████████████████████████| 61kB 7.2MB/s 
     |████████████████████████████████| 798kB 43.6MB/s 
     |████████████████████████████████| 3.0MB 43.6MB/s 
     |████████████████████████████████| 1.1MB 41.9MB/s 
     |████████████████████████████████| 890kB 47.4MB/s 
     |████████████████████████████████| 51kB 7.0MB/s 
     |████████████████████████████████| 655kB 39.4MB/s 
     |████████████████████████████████| 102kB 658kB/s 
ERROR: panel 0.9.7 has requirement bokeh>=2.1, but you'll

In [4]:
import pandas as pd

df = pd.read_csv('dankmemes_task1_train.csv')
embedding = pd.read_csv('dankmemes_task1_train_embeddings.csv', header=None)

In [5]:
import tensorflow as tf
from tensorflow.keras import Model

class MyModel(Model):
  def __init__(self):
    super(MyModel, self).__init__()
    self.dense1 = tf.keras.layers.Dense(1024, activation=tf.keras.activations.relu)
    self.drop = tf.keras.layers.Dropout(rate=0.5)
    self.dense2 = tf.keras.layers.Dense(512, activation=tf.keras.activations.relu)
    self.dense3 = tf.keras.layers.Dense(128, activation=tf.keras.activations.relu)
    self.dense4 = tf.keras.layers.Dense(32, activation=tf.keras.activations.relu)
    self.dense5 = tf.keras.layers.Dense(1, activation=tf.keras.activations.sigmoid) 

  def call(self, x):
    x = self.dense1(x)
    x = self.drop(x)
    x = self.dense2(x)
    x = self.dense3(x)
    x = self.dense4(x)
    return self.dense5(x)

model = MyModel()

In [6]:
loss_object = tf.keras.losses.BinaryCrossentropy(from_logits=False)
optimizer = tf.keras.optimizers.RMSprop()

In [7]:
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.BinaryAccuracy(name='train_accuracy')

test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.BinaryAccuracy(name='test_accuracy')

In [8]:
@tf.function
def train_step(images, labels):
  with tf.GradientTape() as tape:
    predictions = model(images, training=True)
    loss = loss_object(labels, predictions)
  gradients = tape.gradient(loss, model.trainable_variables)
  optimizer.apply_gradients(zip(gradients, model.trainable_variables))

  train_loss(loss)
  train_accuracy(labels, predictions)

In [9]:
@tf.function
def test_step(images, labels):
  predictions = model(images, training=False)
  t_loss = loss_object(labels, predictions)

  test_loss(t_loss)
  test_accuracy(labels, predictions)

In [10]:
def ds_from_df(df, shuffle=True, batch_size=32):
  dataframe = dataframe.copy()
  labels = dataframe.pop('Meme')
  ds = tf.data.Dataset.from_tensor_slices((dict(dataframe), labels))
  if shuffle:
    ds = ds.shuffle(buffer_size=len(dataframe))
  ds = ds.batch(batch_size)
  return ds

In [11]:
import numpy as np

X, y = np.array([embedding[1][i].split() for i in range(1600)]).astype(float), df[['Meme']].values

In [12]:
def create_map():
  temp = df[['Visual']].values
  counter = 0
  map = {}
  for i in temp:
    for j in i[0].split():
      if j.strip(',') not in map and j.strip(',') != '0':
        map[j.strip(',')] = counter
        counter += 1
  return map

In [13]:
def ohe(map):
  one_hot_encode = []
  temp = df[['Visual']].values

  for i in temp:
    arr = list(np.zeros(len(map),dtype=int))
    for j in i[0].split():
      if j.strip(',') != '0':
        arr[map[j.strip(',')]] = 1
    one_hot_encode.append(arr)
  return np.array(one_hot_encode)

In [14]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler, OneHotEncoder
from transformers import BertTokenizer, TFBertModel

def bert(texts):
  res = []
  for text in texts:
    input_ids = tf.constant(tokenizer.encode(text))[None, :]
    res.append(np.array(brt(input_ids)[1]).ravel())
  return np.array(res)

tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-uncased')
brt = TFBertModel.from_pretrained('bert-base-multilingual-uncased')
ssc = StandardScaler()
mms = MinMaxScaler()

Some weights of the model checkpoint at bert-base-multilingual-uncased were not used when initializing TFBertModel: ['nsp___cls', 'mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the model checkpoint at bert-base-multilingual-uncased.
If your task is similar to the task the model of the ckeckpoint was trained on, you can already use TFBertModel for predictions without further training.


In [15]:
import pickle
from datetime import date

temp = mms.fit_transform(np.array([(date(int(i[0].split('-')[0]), int(i[0].split('-')[1]), int(i[0].split('-')[2])) - date(2015, 1, 1)).days for i in df[['Date']].values.tolist()]).reshape(1600, 1))
print(temp.shape)
X = np.hstack((X, temp))
print(X.shape)

temp = ssc.fit_transform(df[['Engagement']].values)
print(temp.shape)
X = np.hstack((X, temp))
print(X.shape)

temp = ohe(create_map())
print(temp.shape)
X = np.hstack((X, temp))
print(X.shape)

temp = pickle.load(open('bert_embeddings.pkl', 'rb'))
print(temp.shape)
X = np.hstack((X, temp))
print(X.shape)

(1600, 1)
(1600, 2049)
(1600, 1)
(1600, 2050)
(1600, 71)
(1600, 2121)
(1600, 768)
(1600, 2889)


In [16]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y)
ds_train = tf.data.Dataset.from_tensor_slices((X_train, y_train))
ds_test = tf.data.Dataset.from_tensor_slices((X_test, y_test))

In [17]:
print(ds_train)
print(ds_test)

<TensorSliceDataset shapes: ((2889,), (1,)), types: (tf.float64, tf.int64)>
<TensorSliceDataset shapes: ((2889,), (1,)), types: (tf.float64, tf.int64)>


In [18]:
buffer_size = 10000
batch_size = 64
num_epochs = 100

In [19]:
ds_train = ds_train.shuffle(buffer_size=buffer_size,
                            reshuffle_each_iteration=False)
ds_test = ds_test.batch(batch_size)
ds_train = ds_train.batch(batch_size)

In [20]:
for epoch in range(100):
  train_loss.reset_states()
  train_accuracy.reset_states()
  test_loss.reset_states()
  test_accuracy.reset_states()

  for features, labels in ds_train:
    train_step(features, labels)

  for test_features, test_labels in ds_test:
    test_step(test_features, test_labels)

  template = 'Epoch {}, Loss: {}, Accuracy: {}, Test Loss: {}, Test Accuracy: {}'
  print(template.format(epoch + 1,
                        train_loss.result(),
                        train_accuracy.result() * 100,
                        test_loss.result(),
                        test_accuracy.result() * 100))


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

Epoch 1, Loss: 1.348555564880371, Accuracy: 55.078125, Test Loss: 0.6158968210220337, Test Accuracy: 71.25
Epoch 2, Loss: 0.6326921582221985, Accuracy: 66.640625, Test Loss: 0.5544939041137695, Test Accuracy: 72.1875
Epoch 3, Loss: 0.5839303731918335, Accuracy: 68.828125, Test Loss: 0.5584436058998108, Test Accuracy: 73.75
Epoch 4, Loss: 0.5754336714744568, Accuracy: 70.546875, Test Loss: 0.5646582841873169, Test Accuracy: 71.25
Epoch 5, Loss: 0.540873110294342, Accuracy: 74.53125, Test Loss: 0.54359358